# VacationPy

In [33]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

In [2]:
# Import weather data
weather_data = pd.read_csv("../WeatherPy/output_data/cities.csv")
weather_data.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Moussoro,13.6408,16.4901,95.83,34,44,12.15,TD,1631971114
1,Kontagora,10.3999,5.4695,84.29,66,100,5.97,NG,1631971115
2,Faanui,-16.4833,-151.7500,77.81,76,16,21.61,PF,1631971117
3,Tripoli,32.8752,13.1875,91.92,35,0,10.42,LY,1631970886
4,Kalmunai,7.4167,81.8167,83.41,75,78,7.81,LK,1631971118


### Humidity Heatmap

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude for locations
locations = weather_data[["Lat", "Lng"]]

# Store humidity for weights
humidity = weather_data["Humidity"]

In [12]:
# Plot Heatmap
fig = gmaps.figure(center=(0, 0), zoom_level=2)

# Store maximum humidity for max_intensity
max_intensity = np.max(humidity)

# Create heat layer and add to figure
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=max_intensity, point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria


In [15]:
narrowed_city_df = weather_data.loc[
    (weather_data["Max Temp"] >= 70) & (weather_data["Max Temp"] <= 80) & 
    (weather_data["Wind Speed"] <= 10) & (weather_data["Cloudiness"] == 0)].dropna()
narrowed_city_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
79,Cape Town,-33.9258,18.4232,71.73,38,0,5.55,ZA,1631970765
86,Seoul,37.5683,126.9778,71.04,83,0,4.61,KR,1631971027
108,Saldanha,-33.0117,17.9442,73.45,51,0,9.26,ZA,1631971202
171,Lata,40.1629,-8.3327,77.07,35,0,9.89,PT,1631971270
175,Shenjiamen,29.9576,122.2980,73.63,74,0,7.58,CN,1631971274
382,Arrondissement de Reims,49.1667,4.0833,74.95,49,0,9.22,FR,1631971498
420,Belgrade,44.8040,20.4651,74.39,46,0,5.75,RS,1631971546
463,Ube,33.9431,131.2511,73.36,83,0,8.05,JP,1631971593
533,Deqing,30.5448,119.9599,75.16,89,0,2.26,CN,1631971656


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [18]:
# Create a dataframe for hotels in the cities
hotel_df = narrowed_city_df.loc[:, ["City","Country", "Lat", "Lng"]]

# Add a new column for the hotel names
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
79,Cape Town,ZA,-33.9258,18.4232,
86,Seoul,KR,37.5683,126.9778,
108,Saldanha,ZA,-33.0117,17.9442,
171,Lata,PT,40.1629,-8.3327,
175,Shenjiamen,CN,29.9576,122.2980,
382,Arrondissement de Reims,FR,49.1667,4.0833,
420,Belgrade,RS,44.8040,20.4651,
463,Ube,JP,33.9431,131.2511,
533,Deqing,CN,30.5448,119.9599,


In [20]:
# Prepare paramaters for API call
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

In [37]:
for index, row in hotel_df.iterrows():
    # Get parameter data from the dataframe
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]

    # Change location of each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Make the call request
    print(f"Retrieving Results for: {city_name}.")
    response = requests.get(base_url, params=params).json()

    # Extract 'results' from the json file
    results = response['results']
    
    # Use try function if no hotel is found
    try:
        # Add to the hotel_df dataframe
        print(f"The closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
        print("------------")
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
print("-------End of Search-------")

Retrieving Results for: Cape Town.
The closest hotel in Cape Town is President Hotel.
------------
Retrieving Results for: Seoul.
The closest hotel in Seoul is Staz Hotel Myeongdong 1.
------------
Retrieving Results for: Saldanha.
The closest hotel in Saldanha is Saldanha Bay Hotel.
------------
Retrieving Results for: Lata.
Missing field/result... skipping.
Retrieving Results for: Shenjiamen.
The closest hotel in Shenjiamen is Pullman Zhoushan Seaview.
------------
Retrieving Results for: Arrondissement de Reims.
The closest hotel in Arrondissement de Reims is Hôtel Première Classe Reims Sud Murigny.
------------
Retrieving Results for: Belgrade.
The closest hotel in Belgrade is Belgrade City Hotel.
------------
Retrieving Results for: Ube.
The closest hotel in Ube is ANA Crowne Plaza Ube, an IHG Hotel.
------------
Retrieving Results for: Deqing.
The closest hotel in Deqing is Crowne Plaza Deqing.
------------
-------End of Search-------


In [38]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
79,Cape Town,ZA,-33.9258,18.4232,President Hotel
86,Seoul,KR,37.5683,126.9778,Staz Hotel Myeongdong 1
108,Saldanha,ZA,-33.0117,17.9442,Saldanha Bay Hotel
171,Lata,PT,40.1629,-8.3327,
175,Shenjiamen,CN,29.9576,122.2980,Pullman Zhoushan Seaview
382,Arrondissement de Reims,FR,49.1667,4.0833,Hôtel Première Classe Reims Sud Murigny
420,Belgrade,RS,44.8040,20.4651,Belgrade City Hotel
463,Ube,JP,33.9431,131.2511,"ANA Crowne Plaza Ube, an IHG Hotel"
533,Deqing,CN,30.5448,119.9599,Crowne Plaza Deqing


In [48]:
# Investigate missing hotel name for Lata
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key,
    "location" : "40.1629,-8.3327"
}
response = requests.get(base_url, params=params).json()
pprint(response)

{'html_attributions': [], 'results': [], 'status': 'ZERO_RESULTS'}


In [49]:
# Remove Lata from the hotel_df
#hotel_df = hotel_df.drop([171]) NOTE: it has already been 
hotel_df

,City,Country,Lat,Lng,Hotel Name
79,Cape Town,ZA,-33.9258,18.4232,President Hotel
86,Seoul,KR,37.5683,126.9778,Staz Hotel Myeongdong 1
108,Saldanha,ZA,-33.0117,17.9442,Saldanha Bay Hotel
175,Shenjiamen,CN,29.9576,122.2980,Pullman Zhoushan Seaview
382,Arrondissement de Reims,FR,49.1667,4.0833,Hôtel Première Classe Reims Sud Murigny
420,Belgrade,RS,44.8040,20.4651,Belgrade City Hotel
463,Ube,JP,33.9431,131.2511,"ANA Crowne Plaza Ube, an IHG Hotel"
533,Deqing,CN,30.5448,119.9599,Crowne Plaza Deqing


In [50]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [51]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))